# RPC Dataset - Data Prepare

In [127]:
import numpy as np
import pandas as pd
import random
import os
from skimage.io import imread
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from matplotlib.patches import Rectangle
from keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
print(os.listdir("RPC_dataset/retail_product_checkout"))

['.DS_Store', 'instances_train2019.json', 'val2019', 'test2019', 'instances_train2019 copy.json', 'train2019.zip', 'test2019.zip', 'export_train_terbaru2.json', 'instances_train2017.json', 'instances_val2019.json', 'Export_train_terbaru.json', 'train2019', 'instances_test2019.json', 'train_candy']


## Train Data

In [128]:
train_json = open("RPC_dataset/retail_product_checkout/instances_train2019.json")
train_data = json.load(train_json)

info_train = pd.DataFrame(train_data['info'])
kategori_train = pd.DataFrame(train_data['categories'])
image_train = pd.DataFrame(train_data['images'])
anotasi_train = pd.DataFrame(train_data['annotations'])
kode_train = pd.DataFrame(train_data['__raw_Chinese_name_df'])

In [129]:
#Kategori khusus tissue
kategori_khusus=kategori_train.drop(kategori_train[(kategori_train['supercategory'] != 'candy')].index)
#agar dimulai dari 0 (.index)
id_khusus=kategori_khusus['id']
id_khusus = id_khusus.reset_index(drop=True)

In [130]:
#ambil khusus id-nya di kodenya
kode1_khusus=pd.DataFrame()
khusus=[]
for i in range(id_khusus.shape[0]):
    khusus=kode_train.drop(kode_train[(kode_train['category_id'] != id_khusus[i])].index)
    kode1_khusus=kode1_khusus.append(khusus)

#reset index
kode1_khusus = kode1_khusus.reset_index(drop=True)

In [131]:
#KODE_train = kode1_khusus.iloc[[0,1,2,3,4]]
KODE_train = kode1_khusus
#Ambil id kategorinya saja
KODE_category_train=KODE_train['category_id']
#ambil sebagian kodenya saja
KODE1_train=KODE_train['code']

In [132]:
KODE_train

,sku_name,category_id,sku_class,code,shelf,num,name,clas,known,ind
0,142_candy,142,candy,6911316101043,4,4,比巴卜棉花泡泡糖可乐味11g,糖果,True,141
1,143_candy,143,candy,6911316101012,4,6,比巴卜棉花泡泡堂葡萄味11g,糖果,True,142
2,144_candy,144,candy,6923450663981,4,4,星爆缤纷原果味25g,糖果,True,143
3,145_candy,145,candy,6911316510005,4,5,阿尔卑斯焦香牛奶味硬糖45g,糖果,True,144
4,146_candy,146,candy,6911316380288,4,5,阿尔卑斯牛奶软糖黄桃酸奶味47g,糖果,False,145
5,147_candy,147,candy,6911316380271,4,5,阿尔卑斯牛奶软糖蓝莓酸奶味47g,糖果,False,146
6,148_candy,148,candy,6901424286213,4,5,王老吉润喉糖28g,糖果,True,147
7,149_candy,149,candy,6907992632483,4,4,伊利牛奶片蓝莓味32g,糖果,True,148
8,150_candy,150,candy,6914782114371,4,5,熊博士口嚼糖草莓牛奶味52g,糖果,True,149
9,151_candy,151,candy,6923450603550,4,5,彩虹糖原果味45g,糖果,True,150


In [133]:
nama_file_train = image_train['file_name']

In [134]:
nama_file = image_train['file_name']
jumlah_gambar = image_train.shape[0]

#menggabungkan kode yang ada menjadi nama file
matches = []
for i in range(len(KODE1_train)):
    matches1 = [match for match in nama_file_train if str(KODE1_train[i]) in match]
    matches = matches + matches1

namafile_pilihan = matches
len(namafile_pilihan) 

3040

In [135]:
#ambil nama file yang diperlukan
image1 = image_train
image2=[]
image3=pd.DataFrame()

for i in range(len(namafile_pilihan)):
    image2 = image1['file_name'] == namafile_pilihan[i]
    image2 = image1[image2]
    image3 = pd.concat([image3,image2])

#ambil idnya
image3_id = image3['id']
#reset index
image3_id = image3_id.reset_index(drop=True)

In [136]:
#Mencari image id-nya pada anotasi train
ano=anotasi_train
ano2=[]
ano3=pd.DataFrame()

for i in range(len(image3_id)):
    ano2 = ano['image_id'] == image3_id[i]
    ano2 = ano[ano2]
    ano3 = pd.concat([ano3,ano2], ignore_index = True)

In [137]:
#ambil data tanpa kolom id-nya
image3_noid = image3[['file_name','height','width']]
#reset index
image3_noid = image3_noid.reset_index(drop=True)

In [138]:
#List data train lengkap
list_data_train=pd.DataFrame()
list_data_train = ano3

list_data_train.insert(0,'file_name',image3_noid['file_name'])
list_data_train.insert(1,'height',image3_noid['height'])
list_data_train.insert(2,'width',image3_noid['width'])

In [139]:
list_data_train

,file_name,height,width,area,bbox,category_id,id,image_id,iscrowd,segmentation,point_xy
0,6911316101043-back_camera0-10.jpg,1944,2592,68590.70,"[1169.05, 1000.25, 441.94, 155.2]",142,13427,13427,0,[[]],"[1390.02, 1077.85]"
1,6911316101043-back_camera0-11.jpg,1944,2592,70638.99,"[1203.24, 1000.25, 462.98, 152.57]",142,13428,13428,0,[[]],"[1434.73, 1076.54]"
2,6911316101043-back_camera0-12.jpg,1944,2592,70380.35,"[1162.52, 1011.42, 545.05, 129.13]",142,13429,13429,0,[[]],"[1435.05, 1075.98]"
3,6911316101043-back_camera0-13.jpg,1944,2592,98350.81,"[1106.28, 1008.33, 640.98, 153.44]",142,13430,13430,0,[[]],"[1426.77, 1085.05]"
4,6911316101043-back_camera0-14.jpg,1944,2592,121348.37,"[1053.3, 1005.51, 720.78, 168.36]",142,13431,13431,0,[[]],"[1413.69, 1089.69]"
...,...,...,...,...,...,...,...,...,...,...,...
3035,6923450603550_camera3-6.jpg,1944,2592,176504.12,"[1085.88, 1188.36, 613.76, 287.58]",151,27660,27660,0,[[]],"[1392.76, 1332.15]"
3036,6923450603550_camera3-4.jpg,1944,2592,196939.79,"[1066.87, 1191.58, 686.77, 286.76]",151,27661,27661,0,[[]],"[1410.25, 1334.96]"
3037,6923450603550_camera3-7.jpg,1944,2592,167656.24,"[1103.84, 1186.36, 557.55, 300.7]",151,27662,27662,0,[[]],"[1382.61, 1336.71]"
3038,6923450603550_camera3-8.jpg,1944,2592,137993.96,"[1134.77, 1194.96, 486.38, 283.72]",151,27664,27664,0,[[]],"[1377.96, 1336.82]"


In [173]:
list_filename_train = list_data_train['file_name']
list_categoryid_train = list_data_train['category_id']-(list_data_train['category_id'][0]-1)
list_bbox_train = list_data_train['bbox']

In [274]:
#ambil nama file yang diperlukan
the_bbox=pd.DataFrame()
label_data_train=pd.DataFrame()
label_data_train2=pd.DataFrame()

nama_bbox=['bbox1','bbox2','bbox3','bbox4']
for i in range(4):
    labels2=pd.DataFrame(columns = ['bbox'])
    for j in range(len(list_bbox_train)):
        labels2.loc[j] = list_bbox_train[j][i]
    label_data_train.insert(i,nama_bbox[i],labels2)


label_data_train.insert(0,'category_id',list_categoryid_train)

In [283]:
#membuat label dengan format txt

a = label_data_train.to_numpy().astype(float)
for i in range(a.shape[0]):
    np.savetxt(r'train/labels/'+ str(list_filename_train[i]) +'.txt', np.array([a[i][:]]).astype(float), fmt='%g')

membuat label dengan format txt

## Validation Data

In [140]:
val_json = open("RPC_dataset/retail_product_checkout/instances_val2019.json")
val_data = json.load(val_json)

info_val = pd.DataFrame(val_data['info'])
kategori_val = pd.DataFrame(val_data['categories'])
image_val = pd.DataFrame(val_data['images'])
anotasi_val = pd.DataFrame(val_data['annotations'])
kode_val = pd.DataFrame(val_data['__raw_Chinese_name_df'])

In [141]:
#Gunakan id khusus yang dipilih dari train untuk validation dataset
anotasi_val1 = anotasi_val
anotasi_val2=[]
anotasi_val3=pd.DataFrame()

for i in range(len(KODE_category_train)):
    anotasi_val2 = anotasi_val1['category_id'] == KODE_category_train[i]
    anotasi_val2 = anotasi_val1[anotasi_val2]
    #image3.loc[len(image3.index)]=image2
    anotasi_val3 = pd.concat([anotasi_val3,anotasi_val2])

anotasi_val3 = anotasi_val3.reset_index(drop=True)

In [303]:
# Pegambilan Kode
#diambil bagian kode untuk citra dan id nya aja
kode1_val=kode_val[['code','ind']]

kode1_khusus=pd.DataFrame()
khusus=[]
for i in range(KODE_category_train.shape[0]):
    khusus=kode1_val.drop(kode1_val[(kode1_val['ind'] != KODE_category_train[i])].index)
    kode1_khusus=kode1_khusus.append(khusus)

#reset index
kode1_khusus_val = kode1_khusus.reset_index(drop=True)
KODE_val = kode1_khusus_val['code']

In [304]:
KODE_val

0    6911316101012
1    6923450663981
2    6911316510005
3    6911316380288
4    6911316380271
5    6901424286213
6    6907992632483
7    6914782114371
8    6923450603550
9    6932107253215
Name: code, dtype: int64

In [142]:
#Ambil bagian id gambar validation mode easy
image_val2 = image_val.drop(image_val[(image_val['level'] != 'easy')].index)
image_val2_easy = image_val2['id']

In [143]:
anotasi_val1 = anotasi_val3
anotasi_val2=[]
anotasi_val3=pd.DataFrame()

for i in range(len(image_val2_easy)):
    anotasi_val2 = anotasi_val1['image_id'] == image_val2_easy[i]
    anotasi_val2 = anotasi_val1[anotasi_val2]
    #image3.loc[len(image3.index)]=image2
    anotasi_val3 = pd.concat([anotasi_val3,anotasi_val2], ignore_index = True)

anotasi_last_val = anotasi_val3

In [144]:
anotasi_val_id = anotasi_last_val['image_id']

In [145]:
#Mengambil nama file berdasarkan data idnya
image1 = image_val
image2=[]
image3=pd.DataFrame()

for i in range(len(anotasi_val_id)):
    image2 = image1['id'] == anotasi_val_id[i]
    image2 = image1[image2]
    image3 = pd.concat([image3,image2], ignore_index = True)

image_val3_noid=image3

In [146]:

list_data_val = anotasi_last_val

list_data_val.insert(0,'file_name',image_val3_noid['file_name'])
list_data_val.insert(1,'height',image_val3_noid['height'])
list_data_val.insert(2,'width',image_val3_noid['width'])
list_data_val.insert(3,'level',image_val3_noid['level'])

In [147]:
list_data_val = list_data_val.sort_values('category_id')
list_data_val = list_data_val.reset_index(drop=True)

In [148]:
list_data_val

,file_name,height,width,level,area,bbox,category_id,id,image_id,iscrowd,segmentation,point_xy
0,20180830-10-00-52-1876.jpg,1855,1855,easy,169379.64,"[797.26, 1331.43, 451.89, 374.82]",142,71682,10048,0,[[]],"[1023.2, 1518.84]"
1,20180830-11-02-57-1916.jpg,1855,1855,easy,153679.21,"[657.52, 663.36, 450.72, 340.96]",142,16576,3206,0,[[]],"[882.88, 833.84]"
2,20180830-11-02-50-1916.jpg,1853,1853,easy,129946.78,"[1105.04, 932.21, 288.49, 450.43]",142,16571,3205,0,[[]],"[1249.28, 1157.42]"
3,20180830-11-02-50-1916.jpg,1853,1853,easy,117035.80,"[906.36, 654.6, 272.16, 430.02]",142,16570,3205,0,[[]],"[1042.44, 869.61]"
4,20180830-11-02-44-1916.jpg,1853,1853,easy,133500.35,"[101.89, 960.39, 303.01, 440.58]",142,16569,3204,0,[[]],"[253.39, 1180.68]"
...,...,...,...,...,...,...,...,...,...,...,...,...
750,20180828-10-29-58-1056.jpg,1850,1850,easy,119607.00,"[1008.69, 697.4, 324.31, 368.8]",151,7412,1641,0,[[]],"[1170.85, 881.8]"
751,20180828-10-29-58-1056.jpg,1850,1850,easy,114602.35,"[199.33, 1245.17, 383.07, 299.17]",151,7416,1641,0,[[]],"[390.87, 1394.76]"
752,20180830-11-02-44-1916.jpg,1853,1853,easy,142137.94,"[499.25, 887.36, 397.75, 357.36]",151,16567,3204,0,[[]],"[698.12, 1066.04]"
753,20180828-10-29-52-1056.jpg,1847,1847,easy,95353.65,"[386.24, 1049.12, 257.07, 370.92]",151,7409,1640,0,[[]],"[514.77, 1234.58]"


In [284]:
list_filename_val = list_data_val['file_name']
list_categoryid_val = list_data_val['category_id']-(list_data_val['category_id'][0]-1)
list_bbox_val = list_data_val['bbox']

In [285]:
#ambil nama file yang diperlukan
the_bbox=pd.DataFrame()
label_data_val=pd.DataFrame()
label_data_val2=pd.DataFrame()

nama_bbox=['bbox1','bbox2','bbox3','bbox4']
for i in range(4):
    labels2=pd.DataFrame(columns = ['bbox'])
    for j in range(len(list_bbox_val)):
        labels2.loc[j] = list_bbox_val[j][i]
    label_data_val.insert(i,nama_bbox[i],labels2)

label_data_val.insert(0,'category_id',list_categoryid_val)

In [290]:
list_filename_val

0      20180830-10-00-52-1876.jpg
1      20180830-11-02-57-1916.jpg
2      20180830-11-02-50-1916.jpg
3      20180830-11-02-50-1916.jpg
4      20180830-11-02-44-1916.jpg
                  ...            
750    20180828-10-29-58-1056.jpg
751    20180828-10-29-58-1056.jpg
752    20180830-11-02-44-1916.jpg
753    20180828-10-29-52-1056.jpg
754    20180828-09-33-25-1021.jpg
Name: file_name, Length: 755, dtype: object

In [289]:
#membuat label dengan format txt

b = label_data_val.to_numpy().astype(float)
for i in range(b.shape[0]):
    np.savetxt(r'val/labels/'+ str(list_filename_val[i]) +'.txt', np.array([b[i][:]]).astype(float), fmt='%g')

In [328]:
# importing the modules
import os
import shutil

# Providing the folder path
origin = 'RPC_dataset/retail_product_checkout/val2019/'
target = 'val/images/'

# Fetching the list of all the files
files = os.listdir(origin)

# Fetching all the files to directory
for i in range(len(list_filename_val)):
    shutil.copy(origin+list_filename_val[i], target+list_filename_val[i])


## Test Data

In [149]:
test_json = open("RPC_dataset/retail_product_checkout/instances_test2019.json")
test_data = json.load(test_json)

info_test = pd.DataFrame(test_data['info'])
kategori_test = pd.DataFrame(test_data['categories'])
image_test = pd.DataFrame(test_data['images'])
anotasi_test = pd.DataFrame(test_data['annotations'])
kode_test = pd.DataFrame(test_data['__raw_Chinese_name_df'])

In [150]:
#Gunakan id khusus yang dipilih dari train untuk validation dataset
anotasi_test1 = anotasi_test
anotasi_test2=[]
anotasi_test3=pd.DataFrame()

for i in range(len(KODE_category_train)):
    anotasi_test2 = anotasi_test1['category_id'] == KODE_category_train[i]
    anotasi_test2 = anotasi_test1[anotasi_test2]
    anotasi_test3 = pd.concat([anotasi_test3,anotasi_test2])

anotasi_test3 = anotasi_test3.reset_index(drop=True)

In [151]:
#Ambil bagian id gambar validation mode easy
image_test2 = image_test.drop(image_test[(image_test['level'] != 'easy')].index)
image_test2_easy = image_test2['id']

In [152]:
anotasi_test1 = anotasi_test3
anotasi_test2=[]
anotasi_test3=pd.DataFrame()

for i in range(len(image_test2_easy)):
    anotasi_test2 = anotasi_test1['image_id'] == image_test2_easy[i]
    anotasi_test2 = anotasi_test1[anotasi_test2]
    #image3.loc[len(image3.index)]=image2
    anotasi_test3 = pd.concat([anotasi_test3,anotasi_test2], ignore_index = True)

anotasi_last_test = anotasi_test3

In [153]:
anotasi_test_id = anotasi_last_test['image_id']

#Mengambil nama file berdasarkan data idnya
image1 = image_test
image2=[]
image3=pd.DataFrame()

for i in range(len(anotasi_test_id)):
    image2 = image1['id'] == anotasi_test_id[i]
    image2 = image1[image2]
    image3 = pd.concat([image3,image2], ignore_index = True)

image_test3_noid=image3

In [154]:
list_data_test = anotasi_last_test

list_data_test.insert(0,'file_name',image_test3_noid['file_name'])
list_data_test.insert(1,'height',image_test3_noid['height'])
list_data_test.insert(2,'width',image_test3_noid['width'])
list_data_test.insert(3,'level',image_test3_noid['level'])

In [155]:
list_data_test = list_data_test.sort_values('category_id')
list_data_test = list_data_test.reset_index(drop=True)

In [156]:
list_data_test

,file_name,height,width,level,area,bbox,category_id,id,image_id,iscrowd,segmentation,point_xy
0,20180830-10-01-56-1877.jpg,1855,1855,easy,182396.31,"[425.38, 1242.51, 411.61, 443.13]",142,46156,7258,0,[[]],"[631.18, 1464.08]"
1,20180828-10-59-37-819.jpg,1815,1815,easy,160696.60,"[471.31, 745.22, 376.34, 427.0]",142,58411,8649,0,[[]],"[659.48, 958.72]"
2,20180828-10-59-50-819.jpg,1815,1815,easy,138686.05,"[444.82, 719.24, 363.24, 381.8]",142,58423,8650,0,[[]],"[626.44, 910.14]"
3,20180828-11-00-09-819.jpg,1802,1802,easy,138802.49,"[731.04, 765.17, 384.42, 361.07]",142,58432,8651,0,[[]],"[923.25, 945.7]"
4,20180828-11-18-28-833.jpg,1812,1812,easy,176533.60,"[560.2, 859.89, 415.0, 425.38]",142,58497,8658,0,[[]],"[767.7, 1072.58]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3234,20180904-16-34-05-3279.jpg,1835,1835,easy,83606.80,"[441.48, 1215.0, 352.17, 237.4]",151,24753,4428,0,[[]],"[617.57, 1333.7]"
3235,20180828-13-30-49-1127.jpg,1848,1848,easy,133314.32,"[625.86, 634.67, 338.42, 393.94]",151,38976,6361,0,[[]],"[795.07, 831.64]"
3236,20180828-13-30-42-1127.jpg,1847,1847,easy,99147.51,"[375.18, 996.4, 268.65, 369.06]",151,38972,6360,0,[[]],"[509.5, 1180.93]"
3237,20180828-13-18-45-1115.jpg,1832,1832,easy,129079.45,"[768.93, 1412.32, 392.91, 328.53]",151,38917,6353,0,[[]],"[965.38, 1576.59]"


# Preprocessing Data

# Training Data